<a href="https://colab.research.google.com/github/SpatialeLearning/ee-python-colab/blob/main/03_CART_Landsat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **CART Supervised Classification**

**Authenticate and Initialize Earth Engine**

In [13]:
#!pip install geemap

In [4]:
#Import libraries.
import ee
import geemap

In [14]:
#Authenticate and initialize ee
#ee.Authenticate()
#ee.Initialize()

In [10]:
Map = geemap.Map()

In [6]:
# CART Supervised Classification with Landsat

#ROI
roi = ee.Geometry.Point([-122.26, 37.87])

# Load Landsat 8 data
image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
.filterDate('2017-01-01', '2017-12-31')\
.filterBounds(roi)\
.sort('CLOUD_COVER')\
.first()
          
visParamsTrue = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000, 'gamma': 1.4}

In [7]:
# Import training data
training = ee.FeatureCollection('users/midekisa/Train_Cover_CA')

label = 'Class'
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B7']
input = image.select(bands)

# Overlay the points on the imagery to get training.
trainImage = input.sampleRegions(**{
  'collection': training,
  'properties': [label],
  'scale': 30
})

trainingData = trainImage.randomColumn()
trainSet = trainingData.filter(ee.Filter.lessThan('random', 0.8))
testSet = trainingData.filter(ee.Filter.greaterThanOrEquals('random', 0.8))


In [8]:
# Classification Model
classifier = ee.Classifier.smileCart().train(trainSet, label, bands)

# Classify the image
classified = input.classify(classifier)
#print(classified.getInfo())

# Define a palette for the classification.
landcoverPalette = [
  '#0c2c84', #water (0)
  '#e31a1c', #urban (1)
  '#005a32', #forest (2)
  'FF8000', #crop (3)
  '969696', #barren (4)
]

In [12]:
# Add the layer to the map object.
Map.addLayer(image, visParamsTrue, "Landsat 2017")
Map.addLayer(classified, {'palette': landcoverPalette, 'min': 0, 'max':4 }, 'classification')
Map.centerObject(image, 8)
Map 

Map(bottom=754.0, center=[37.47294190468172, -122.11358150908953], controls=(WidgetControl(options=['position'…